In [1]:
!which python

/home/ec2-user/anaconda3/envs/python3/bin/python


In [2]:
!conda install cudatoolkit=10.0 -y
!pip install git+https://github.com/allenai/longformer.git

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::tqdm==4.62.3=pyhd8ed1ab_0
  - conda-forge/noarch::black==21.11b1=pyhd8ed1ab_0
  - conda-forge/linux-64::conda-package-handling==1.7.3=py38h497a2fe_1
  - conda-forge/noarch::dask-core==2021.11.2=pyhd8ed1ab_0
  - conda-forge/noarch::imageio==2.9.0=py_0
  - conda-forge/linux-64::pytest==6.2.5=py38h578d9bd_1
  - conda-forge/linux-64::watchdog==2.1.6=py38h578d9bd_1
  - conda-forge/linux-64::aiohttp==3.8.1=py38h497a2fe_0
  - conda-forge/linux-64::astropy==5.0=py38h6c62de6_0
  - conda-forge/linux-64::bokeh==2.4.2=py38h578d9bd_0
  - conda-forge/linux-64::distributed==2021.11.2=py38h578d9bd_0
  - conda-forge/noarch::flask==2.0.2=pyhd8ed1ab_0
  - conda-forge/linux-64::matplotlib-base==3.5.0=py38hf4fb855_0
  - conda-forge/noarch::nbformat==5.1.3=pyhd8ed1ab_0
  - conda-forge/noarch::pylint==2.12.1=pyhd8ed1ab_0
  - conda-forg

In [3]:
import torch
from longformer.longformer import Longformer, LongformerConfig
from longformer.sliding_chunks import pad_to_window_size
from transformers import RobertaTokenizer

config = LongformerConfig.from_pretrained('longformer-base-4096/') 
# choose the attention mode 'n2', 'tvm' or 'sliding_chunks'
# 'n2': for regular n2 attantion
# 'tvm': a custom CUDA kernel implementation of our sliding window attention
# 'sliding_chunks': a PyTorch implementation of our sliding window attention
config.attention_mode = 'sliding_chunks'

model = Longformer.from_pretrained('longformer-base-4096/', config=config)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
tokenizer.model_max_length = model.config.max_position_embeddings

SAMPLE_TEXT = ' '.join(['Hello world! '] * 1000)  # long input document

input_ids = torch.tensor(tokenizer.encode(SAMPLE_TEXT)).unsqueeze(0)  # batch of size 1

# TVM code doesn't work on CPU. Uncomment this if `config.attention_mode = 'tvm'`
# model = model.cuda(); input_ids = input_ids.cuda()

# Attention mask values -- 0: no attention, 1: local attention, 2: global attention
attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=input_ids.device) # initialize to local attention
attention_mask[:, [1, 4, 21,]] =  2  # Set global attention based on the task. For example,
                                     # classification: the <s> token
                                     # QA: question tokens

# padding seqlen to the nearest multiple of 512. Needed for the 'sliding_chunks' attention
input_ids, attention_mask = pad_to_window_size(
        input_ids, attention_mask, config.attention_window[0], tokenizer.pad_token_id)

output = model(input_ids, attention_mask=attention_mask)[0]

In [4]:
!pip install datasets 

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [ ]:
!python longformer/scripts/summarization_opt.py\
--model_path 'longformer-encdec-base-16384'\
--tokenizer 'longformer-encdec-base-16384'\
--epochs 1\
--max_input_len 4000 \
--batch_size 1\
--dataset_name "test"\
--train_file 'stary/model_b_data/train.csv'\
--test_file 'stary/model_b_data/test.csv'\
--validation_file 'stary/model_b_data/test.csv'

I0722 08:09:27.267873 139690818238272 utils.py:157] NumExpr defaulting to 8 threads.
I0722 08:09:27.468686 139690818238272 file_utils.py:39] PyTorch version 1.12.0+cu102 available.
W0722 08:09:33.643760 139690818238272 builder.py:463] Using custom data configuration default-2e5c898ace65ed7d
W0722 08:09:33.645892 139690818238272 builder.py:641] Reusing dataset csv (/home/ec2-user/.cache/huggingface/datasets/csv/default-2e5c898ace65ed7d/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)
100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 922.50it/s]
Namespace(adafactor=False, attention_dropout=0.1, attention_mode='sliding_chunks', attention_window=512, batch_size=1, dataset_name='test', debug=False, disable_checkpointing=False, epochs=1, fp32=False, from_pretrained=None, gpus=-1, grad_accum=1, grad_ckpt=False, label_smoothing=0.0, lr=3e-05, max_input_len=4000, max_output_len=256, model_path='longformer-encdec-base-16384', no_progress_bar=False, num_w

In [44]:
import torch 
from longformer import LongformerEncoderDecoderForConditionalGeneration, LongformerEncoderDecoderConfig
from transformers import AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#config = LongformerEncoderDecoderConfig.from_pretrained('/home/ec2-user/SageMaker/summarization/test/_ckpt_epoch_0_v0.ckpt')
config = LongformerEncoderDecoderConfig.from_pretrained('longformer-encdec-base-16384')
#config.attention_dropout = self.args.attention_dropout
#config.gradient_checkpointing = self.args.grad_ckpt
config.attention_mode = 'sliding_chunks'
#config.attention_window = [self.args.attention_window] * config.encoder_layers

model.model = LongformerEncoderDecoderForConditionalGeneration.from_pretrained('longformer-encdec-base-16384',config = config)
#ckpt_pth = '/home/ec2-user/SageMaker/summarization/test/_ckpt_epoch_0_v2.ckpt'
#x = torch.load(ckpt_pth)['state_dict']
#x['model.model.final_logits_bias'] = x['model.final_logits_bias']

model.load_state_dict(new_state_dict)
#model = torch.load(ckpt_pth,map_location=device)
model.eval()

tokenizer = AutoTokenizer.from_pretrained('longformer-encdec-base-16384')

tokenizer.model_max_length = 4000

In [63]:

SAMPLE_TEXT = ' '.join(['Hello world! '] * 200)  # long input document

input_ids = torch.tensor(tokenizer.encode(SAMPLE_TEXT)).unsqueeze(0)

attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=input_ids.device)
attention_mask[input_ids == tokenizer.pad_token_id] = 0
attention_mask[:, 0] = 2 
half_padding_mod = model.config.attention_window[0]
input_ids, attention_mask = pad_to_window_size(  # ideally, should be moved inside the LongformerModel
                input_ids, attention_mask, half_padding_mod, tokenizer.pad_token_id)



In [65]:
generated_ids =  model.model.generate(input_ids=input_ids, attention_mask=attention_mask,
                                            use_cache=True, max_length=256,
                                            num_beams=1)

In [67]:
generated_str = tokenizer.batch_decode(generated_ids.tolist(), skip_special_tokens=True)

In [68]:
generated_str

['Bro said the sentence']